In [2]:
import os, io, json, argparse, textwrap, time
from typing import Dict, Any
from pptx import Presentation
from pptx.util import Inches
from PIL import Image, ImageDraw, ImageFont
import jsonschema

# Try import OpenAI client (DeepSeek uses OpenAI-compatible API)
try:
    from openai import OpenAI
except Exception:
    OpenAI = None

# ---- JSON schema for slide_spec ----
SLIDE_SPEC_SCHEMA = {
    "type": "object",
    "required": ["title", "slides"],
    "properties": {
        "title": {"type": "string"},
        "slides": {
            "type": "array",
            "items": {
                "type": "object",
                "required": ["type", "title"],
                "properties": {
                    "type": {"type": "string", "enum": ["title_slide", "bullet_slide", "image_slide", "two_column"]},
                    "title": {"type": "string"},
                    "subtitle": {"type": "string"},
                    "bullets": {"type": "array", "items": {"type": "string"}},
                    "image_prompt": {"type": "string"},
                    "notes": {"type": "string"}
                }
            }
        }
    }
}

# ---- Helper: prompt -> call DeepSeek (OpenAI-compatible) ----
def call_deepseek_llm(user_prompt: str, model: str = "deepseek-chat", temperature: float = 0.2) -> Dict[str, Any]:
    """
    使用 OpenAI Python SDK（OpenAI class）调用 DeepSeek（需要 DEEPSEEK_API_KEY 环境变量）。
    返回解析后的 slide_spec dict。
    """
    if OpenAI is None:
        raise RuntimeError("openai package not available in this environment.")
    api_key = os.environ.get("DEEPSEEK_API_KEY")
    if not api_key:
        raise RuntimeError("环境变量 DEEPSEEK_API_KEY 未设置。")

    client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")
    system_prompt = (
        "You are a slide-authoring assistant. Given the user's request, output ONLY a valid JSON object matching the schema:\n"
        '{ "title": "string", "slides": [{"type":"title_slide|bullet_slide|image_slide|two_column", "title":"string", "subtitle?":"string", "bullets?":["..."], "image_prompt?":"string", "notes?":"string"}] }\n'
        "Constraints: max 10 slides. Bullets should be concise. Output STRICT JSON only (no markdown)."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    resp = client.chat.completions.create(model=model, temperature=temperature, messages=messages, stream=False)
    # try to extract JSON-like substring
    raw = resp.choices[0].message.content
    json_text = extract_json_like(raw)
    return json.loads(json_text)

def extract_json_like(s: str) -> str:
    """从模型的返回文本中抽取第一个 { ... } JSON 块；若失败则抛异常。"""
    s = s.strip()
    # remove triple-fence if present
    if s.startswith("```"):
        parts = s.split("```")
        for p in parts:
            p = p.strip()
            if p.startswith("{"):
                s = p
                break
    start = s.find("{")
    end = s.rfind("}")
    if start == -1 or end == -1:
        raise ValueError("无法从模型输出中提取 JSON。原始输出片段:\n" + s[:1000])
    return s[start:end+1]


# ---- Helper: image generation (mock PIL) ----
def generate_placeholder_image(prompt: str, size=(1280,720)) -> bytes:
    """用 PIL 生成带 prompt 文本的占位图，返回 PNG bytes"""
    W,H = size
    img = Image.new("RGB", (W,H), color=(245,245,245))
    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except Exception:
        font = ImageFont.load_default()
    text = "Image placeholder\n" + (prompt or "")
    lines = textwrap.wrap(text, width=60)
    y = 40
    for line in lines:
        draw.text((40,y), line, font=font, fill=(30,30,30))
        y += 30
    draw.rectangle([8,8,W-8,H-8], outline=(200,200,200), width=2)
    bio = io.BytesIO()
    img.save(bio, format="PNG")
    return bio.getvalue()


# ---- Helper: render PPT with python-pptx ----
def render_pptx_from_spec(spec: Dict[str, Any], out_path: str = "output.pptx"):
    """根据 slide_spec 生成 pptx 并保存"""
    # basic presentation (if you have a custom template, load it)
    prs = Presentation()

    for s in spec.get("slides", []):
        stype = s.get("type", "bullet_slide")
        if stype == "title_slide":
            layout = prs.slide_layouts[0] if len(prs.slide_layouts)>0 else prs.slide_layouts[1]
            slide = prs.slides.add_slide(layout)
            if slide.shapes.title:
                slide.shapes.title.text = s.get("title","")
            if s.get("subtitle"):
                slide.shapes.add_textbox(Inches(1), Inches(1.6), Inches(8), Inches(0.8)).text_frame.text = s.get("subtitle")

        elif stype == "bullet_slide":
            layout = prs.slide_layouts[1] if len(prs.slide_layouts)>1 else prs.slide_layouts[0]
            slide = prs.slides.add_slide(layout)
            try:
                slide.shapes.title.text = s.get("title","")
            except Exception:
                pass
            # body: prefer placeholder 1
            body_tf = None
            try:
                body_tf = slide.shapes.placeholders[1].text_frame
                body_tf.clear()
            except Exception:
                tb = slide.shapes.add_textbox(Inches(1), Inches(1.6), Inches(6), Inches(4))
                body_tf = tb.text_frame
                body_tf.clear()
            for b in s.get("bullets", []):
                p = body_tf.add_paragraph()
                p.text = b
                p.level = 0
            if s.get("image_prompt"):
                img_bytes = generate_placeholder_image(s["image_prompt"])
                slide.shapes.add_picture(io.BytesIO(img_bytes), Inches(6.0), Inches(1.6), width=Inches(3.2))
        elif stype == "two_column":
            layout = prs.slide_layouts[6] if len(prs.slide_layouts)>6 else prs.slide_layouts[1]
            slide = prs.slides.add_slide(layout)
            try:
                slide.shapes.title.text = s.get("title","")
            except Exception:
                pass
            left_tb = slide.shapes.add_textbox(Inches(0.6), Inches(1.6), Inches(4.2), Inches(4))
            left_tf = left_tb.text_frame
            left_tf.clear()
            for b in s.get("bullets", []):
                p = left_tf.add_paragraph()
                p.text = b

        elif stype == "image_slide":
            slide = prs.slides.add_slide(prs.slide_layouts[6] if len(prs.slide_layouts)>6 else prs.slide_layouts[0])
            try:
                slide.shapes.title.text = s.get("title","")
            except Exception:
                pass

            if s.get("bullets"):
                tf = slide.shapes.add_textbox(Inches(0.6), Inches(5.1), Inches(9.0), Inches(1.2)).text_frame
                tf.clear()
                for b in s.get("bullets", []):
                    p = tf.add_paragraph()
                    p.text = b
        else:
            # fallback simple text slide
            slide = prs.slides.add_slide(prs.slide_layouts[1] if len(prs.slide_layouts)>1 else prs.slide_layouts[0])
            try:
                slide.shapes.title.text = s.get("title","")
            except Exception:
                pass
            if s.get("notes"):
                tb = slide.shapes.add_textbox(Inches(1), Inches(1.6), Inches(8), Inches(2))
                tb.text_frame.text = s.get("notes","")

    prs.save(out_path)
    print(f"[DONE] Saved PPT: {out_path}")


# ---- Main orchestration ----
def main():
    p = argparse.ArgumentParser()
    p.add_argument("--prompt", type=str, required=False, default="", help="User prompt for PPT content")
    p.add_argument("--out", type=str, default="output.pptx", help="Output pptx path")
    p.add_argument("--model", type=str, default="deepseek-chat", help="Model name for DeepSeek")

    # Use parse_known_args to avoid ipykernel's extra args breaking argparse when running inside Jupyter
    args, unknown = p.parse_known_args()


    prompt = args.prompt.strip()
    if not prompt:
        # interactive if no prompt provided
        try:
            prompt = input("Enter a short prompt describing the PPT you want: ").strip()
        except Exception:
            prompt = ""
        if not prompt:
            print("No prompt provided, exiting.")
            return

    # choose LLM
    spec = None

    print("[INFO] Calling DeepSeek API to generate slide spec...")
    try:
        spec = call_deepseek_llm(prompt, model=args.model, temperature=0.2)
    except Exception as e:
        print("[WARN] DeepSeek call failed:", e)


    # Validate spec
    try:
        jsonschema.validate(instance=spec, schema=SLIDE_SPEC_SCHEMA)
    except Exception as e:
        print("[ERROR] slide_spec 验证失败:", e)
        print("slide_spec 内容:\n", json.dumps(spec, ensure_ascii=False, indent=2))

    # Render PPT (images are placeholder; replace generate_placeholder_image with real image API if desired)
    render_pptx_from_spec(spec, out_path=args.out)


if __name__ == "__main__":
    main()

Enter a short prompt describing the PPT you want:  Create a PPT of about 10 pages to explain the background, principles, connections, applications, and future development directions of personalized large models and generative recommendation technologies. In Chinese.


[INFO] Calling DeepSeek API to generate slide spec...
[DONE] Saved PPT: output.pptx
